<a href="https://colab.research.google.com/github/nalgo-intern/team-a/blob/master/%E3%83%9D%E3%82%B1%E3%83%A2%E3%83%B3%E3%82%BF%E3%82%A4%E3%83%97%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#numpyのVerが古くないと走らない

In [0]:
!pip install numpy==1.16.

#githubからデータを取得

In [0]:
!git clone https://github.com/nalgo-intern/team-b.git
%cd team-b/learning_program
!mkdir learned_data

#データセットを作成

In [0]:
from PIL import Image
import os, glob
import numpy as np
import random, math

root_dir = "../pictures"
types =["normal", "fire", "water", "grass", "electric", "ice", "fighting", "poison", "ground", "flying", "psychic", "bug", "rock", "dragon"]

X = []
Y = []

def make_sample(files):
    global X, Y
    X = []
    Y = []
    for cat, fname in files:
        add_sample(cat, fname)
    return np.array(X), np.array(Y)


def add_sample(cat, fname):
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((240, 240))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

allfiles = []

for idx, cat in enumerate(types):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))

random.shuffle(allfiles)
th = math.floor(len(allfiles) * 0.8)
train = allfiles[0:th]
test = allfiles[th:]
X_train, y_train = make_sample(train)
X_test, y_test = make_sample(test)
xy = (X_train, X_test, y_train, y_test)

np.save("./dataset.npy", xy)

#学習

In [0]:
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

num_classes = 14
im_rows = 240
im_cols = 240
in_shape = (im_rows, im_cols, 3)

# データを読み込む --- (*1)
X_train, X_test, y_train, y_test = np.load('./dataset.npy')

# データを正規化 --- (*2)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# ラベルデータをOne-Hot形式に変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# モデルを定義 --- (*3)
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=in_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# モデルをコンパイル --- (*4)
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# 学習を実行 --- (*5)
hist = model.fit(X_train, y_train,
    batch_size=32, epochs=50,
    verbose=1,
    validation_data=(X_test, y_test))

# モデルを評価 --- (*6)
score = model.evaluate(X_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])

# 学習の様子をグラフへ描画 --- (*7)
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#モデル・重みの保存
json_string = model.model.to_json()
open('./learned_data/data.json', 'w').write(json_string)

hdf5_file = "./learned_data/data.hdf5"
model.model.save_weights(hdf5_file)

#テスト用データのアップロード  
'test.jpg'というファイル名にすること

In [0]:
from google.colab import files
uploaded = files.upload()

#判別用プログラム  
webアプリに組み込めるように関数化済み

In [0]:
def main(img_path):    
    from keras import models
    import numpy as np
    from keras.preprocessing import image
    from keras.models import model_from_json

    types =["normal", "fire", "water", "grass", "electric", "ice", "fighting", "poison", "ground", "flying", "psychic", "bug", "rock", "dragon"]
    types_jp = ["ノーマル", "ほのお", "みず", "くさ", "でんき", "こおり", "かくとう", "どく", "じめん", "ひこう", "エスパー", "むし", "いわ", "ドラゴン"]

    model = model_from_json(open('./learned_data/data.json').read())
    model.load_weights('./learned_data/data.hdf5')

    img_path = img_path 
    img = image.load_img(img_path, target_size=(240,240,3))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    features = model.predict(x)

    for i in range(0,14):
        if features[0,i] == 1:
            break
    
    return types_jp[i]


if __name__ == '__main__':
    result = main('test.jpg')
    print(result)

#以下GDriveへ学習済みデータをバックアップ

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp -R /content/team-b/learning_program/learned_data /content/drive/My\ Drive